In [23]:
import requests
import json
from Config import api_key

# Get list of all files

In [77]:
from IPython.display import clear_output
file_paths = []


empty = 0
for i in range(1, 10000):
    url = f'https://cchdo.ucsd.edu/api/v1/file/{i}'
    re = requests.get(url, headers = {"X-Authentication-Token": api_key})
    file_json = json.loads(re.text)
    if file_json['file_path'] == '':
        empty += 1
    else:
        file_paths.append(file_json['file_path'])
    clear_output(wait=True)
    print(str(i) + ' - ' +  str(empty))

2311 - 0


KeyboardInterrupt: 

In [79]:
file_paths

['/data/1/06BE200305_nc_hyd.zip',
 '/data/2/pr06_b_ct1.zip.asc',
 '/data/3/32MW051_1trk.gif',
 '/data/4/ar08_c_hy1.csv',
 '/data/5/p01c_1999act.zip',
 '/data/6/32MW071_1_hy.txt',
 '/data/7/archive_2010.tar',
 '/data/8/33H420091207trk.jpg',
 '/data/9/ar28_74CH19850120_ct1.zip',
 '/data/10/a16s_318MSAVE5trk.jpg',
 '/data/11/64TR19900417_hy1.csv',
 '/data/12/AR07W_2002do.pdf',
 '/data/13/33KB20111218trk.jpg',
 '/data/14/06AQ19860627_ct1.zip',
 '/data/15/s04pdo.pdf',
 '/data/16/pr32_316N138_8trk.gif',
 '/data/17/archive_2418.tar',
 '/data/18/ais01_09AR20010101_hy1.csv',
 '/data/19/pr14_e_inv_ctd.txt',
 '/data/20/sr01_dct.zip',
 '/data/21/pr16_q_nc_hyd.zip',
 '/data/22/pr16_20061031su.txt',
 '/data/23/33KB194_1_trk.jpg',
 '/data/24/18HU20110506su.txt',
 '/data/25/35LL19910103su.txt',
 '/data/26/whots4.tar.gz',
 '/data/27/pr15_33KM200311_1trk.gif',
 '/data/28/ar07ea_inv_ctd.txt',
 '/data/29/aaiw05_nc_hyd.zip',
 '/data/30/pr24_a_nc_ctd.zip',
 '/data/31/33KB187_1_trk.jpg',
 '/data/32/ar11_jdo.

In [25]:
#Get list of all cruises and their ids
url = 'https://cchdo.ucsd.edu/api/v1/file/all'
re = requests.get(url, headers = {"X-Authentication-Token": api_key})
raw_json = json.loads(re.text)

In [52]:
raw_json

[{'container_contents': [],
  'cruises': [726],
  'data_container': 'zip',
  'data_format': 'whp_netcdf',
  'data_type': 'bottle',
  'events': [],
  'file_hash': '2fe9e3a0f24e1f12a2500254dbbbbe441415d397409e933e625990a51bd590fd',
  'file_name': '06BE200305_nc_hyd.zip',
  'file_path': '',
  'file_size': 182381,
  'file_sources': [],
  'file_type': 'application/zip',
  'id': 1,
  'other_roles': [],
  'permissions': [],
  'role': 'dataset',
  'submissions': []},
 {'container_contents': [],
  'cruises': [272],
  'data_container': '',
  'data_format': '',
  'data_type': '',
  'events': [],
  'file_hash': '80b407af3fbd2f86eabe2770ebf9a1712e5427f3dc48068e9ac2c94db627d46a',
  'file_name': 'pr06_b_ct1.zip.asc',
  'file_path': '',
  'file_size': 347660,
  'file_sources': [],
  'file_type': 'application/pgp-signature',
  'id': 2,
  'other_roles': [],
  'permissions': [],
  'role': 'merged',
  'submissions': []},
 {'container_contents': [],
  'cruises': [630],
  'data_container': '',
  'data_forma

### Roles, data formats, and data types

Is there a specific subset of this we want to check?
Currently only pulling files where the `data_type` is `bottle` or `ctd`

In [29]:
set([x['role'] for x in raw_json])

{'',
 'ancillary',
 'archive',
 'dataset',
 'hidden',
 'intermediate',
 'merged',
 'product',
 'raw',
 'unprocessed'}

In [51]:
set([x['data_format'] for x in raw_json])

{'',
 'cf_netcdf',
 'exchange',
 'hrp_netcdf',
 'joa_binary',
 'matlab',
 'ods',
 'pdf',
 'sbe_ascii_cnv',
 'sbe_hex_xmlcon_24hz',
 'text',
 'whp_netcdf',
 'woce'}

In [31]:
set([x['data_type'] for x in raw_json])

{'',
 'bottle',
 'ctd',
 'documentation',
 'float',
 'hrp',
 'large_volume',
 'summary',
 'trace_metals'}

In [66]:
empty_path_count = 0
for entry in raw_json:
    if entry['file_path'] == '':
        empty_path_count += 1
empty_path_count

17373

In [68]:
len(raw_json)

18749

# Select bottle/ctd files

In [33]:
to_download = []
for entry in raw_json:
    if entry['data_type'] == 'bottle' or entry['data_type'] == 'ctd':
        to_download.append(entry)

In [46]:
#Check total size to download
total_size = 0
for entry in to_download:
    total_size += int(entry['file_size'])
    
print(str(round(total_size / 1000000000, 2)) + ' GB')

14.84 GB


Most files (8683/8798) don't have a path, what to do about these files?

In [69]:
empty_path_count = 0
for entry in to_download:
    if entry['file_path'] == '':
        empty_path_count += 1
empty_path_count

2941

In [70]:
len(to_download)

2952

In [59]:
for entry in to_download:
    if entry['file_path']:
        file_name = entry['file_name']
        file_path = entry['file_path']
        
        url = f'https://cchdo.ucsd.edu{file_path}'
        re = requests.get(url, allow_redirects=True, headers = {"X-Authentication-Token": api_key})
        open('data/' + file_name, 'wb').write(re.content)

In [62]:
to_download

[{'container_contents': [],
  'cruises': [726],
  'data_container': 'zip',
  'data_format': 'whp_netcdf',
  'data_type': 'bottle',
  'events': [],
  'file_hash': '2fe9e3a0f24e1f12a2500254dbbbbe441415d397409e933e625990a51bd590fd',
  'file_name': '06BE200305_nc_hyd.zip',
  'file_path': '',
  'file_size': 182381,
  'file_sources': [],
  'file_type': 'application/zip',
  'id': 1,
  'other_roles': [],
  'permissions': [],
  'role': 'dataset',
  'submissions': []},
 {'container_contents': [],
  'cruises': [1006],
  'data_container': '',
  'data_format': 'exchange',
  'data_type': 'bottle',
  'events': [],
  'file_hash': '000f57cd7f9296f9c4e5ec633efdf7783df9135535fbec6c3ea2e81073ddb774',
  'file_name': 'ar08_c_hy1.csv',
  'file_path': '',
  'file_size': 31661,
  'file_sources': [],
  'file_type': 'text/csv',
  'id': 4,
  'other_roles': [],
  'permissions': [],
  'role': 'dataset',
  'submissions': []},
 {'container_contents': [],
  'cruises': [21],
  'data_container': 'zip',
  'data_format': 